# Predicting Home Sale Price

This will attempt to be similar to the Zestimate used on Zillow, the Redfin estimate used on Redfin, or any of the valuations provided by Collateral Analytics, CoreLogic, or Quantarium on Realtor.com.

I will create a regression model and use it to predict valuation and compare to the other valuations on currently listed homes.

In [85]:
%run src/imports.py
from src.helpers import calculate_five_percent, calculate_ten_percent

ImportError: cannot import name 'calculate_ten_percent' from 'src.helpers' (/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/src/helpers.py)

In [82]:
sns.set(rc={'figure.figsize':(16,4)})

In [37]:
neighborhoods_cleaned = pd.read_pickle('data/neighborhoods_cleaned.pkl')
neighborhoods_cleaned.reset_index(inplace=True)
neighborhoods_cleaned.drop(['index','DAYS ON MARKET','PROPERTY TYPE','ADDRESS','CITY','STATE OR PROVINCE'],axis=1,inplace=True)
#neighborhoods_cleaned = neighborhoods_cleaned.merge(pd.get_dummies(neighborhoods_cleaned['LOCATION'],drop_first=True), how='outer',left_index=True,right_index=True)
neighborhoods_cleaned.drop(['LOCATION'],axis=1,inplace=True)
neighborhoods_cleaned['SOLD DATE'] = pd.to_datetime(neighborhoods_cleaned['SOLD DATE'])
neighborhoods_cleaned['MONTH'] = pd.DatetimeIndex(neighborhoods_cleaned['SOLD DATE']).month
neighborhoods_cleaned['YEAR'] = pd.DatetimeIndex(neighborhoods_cleaned['SOLD DATE']).year
neighborhoods_cleaned.drop(['SOLD DATE'],axis=1,inplace=True)
neighborhoods_cleaned.drop(['MLS#','SALE TYPE','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)','SOURCE','FAVORITE','INTERESTED'], axis=1,inplace=True)
neighborhoods_cleaned.dropna(inplace=True)

In [38]:
neighborhoods_cleaned.to_pickle('data/modeldata.pkl')

In [39]:
modeldata = neighborhoods_cleaned.copy()

-----
Data Density  
= 11,654 / 11
= 1000
-----

## Building a Model

Smaller set:

In [40]:
df_small = modeldata.sample(1000)

In [41]:
y = df_small['PRICE'].values
X = df_small.drop('PRICE',axis=1).values

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [43]:
rfr = RandomForestRegressor()
rfr.fit(X_train,y_train)
y_predict = rfr.predict(X_test)
print("score:", rfr.score(X_test, y_test))
print('MSE: {:2e}'.format(mean_squared_error(y_test, y_predict)))

score: 0.8543774000704493
MSE: 1.422583e+09


In [73]:
calculate_five_percent(y_predict,y_test)

48.8

In [86]:
search_space = [{'regressor__alpha': 10 ** (np.linspace(-5, 5, 10)), 
                 'regressor__l1_ratio': np.linspace(0,1,6)}]

# l_lasso / (l_lasso + l_ridge)
# if l_lasso = 0: 0 / 1 = 0 (this is ridge regression)
# if l_ridge = 0: l_lasso / l_lasso = 1 (this is LASSSO regression)

pipe = Pipeline([('scaler', StandardScaler()), ('regressor', ElasticNet())])

enet_cv = GridSearchCV(pipe, 
                      search_space, 
                      cv=KFold(10, shuffle=True), 
                      scoring='neg_mean_squared_error',
                      return_train_score = True)

In [87]:
enet_cv.fit(X_train,y_train)

enet_cv.best_params_

/Users/randallbloomquist/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1397526595922.1372, tolerance: 1186125044.7488189
  model = cd_fast.enet_coordinate_descent(
/Users/randallbloomquist/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1417597532078.9778, tolerance: 1159862609.8198588
  model = cd_fast.enet_coordinate_descent(
/Users/randallbloomquist/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1355913933669.8506, tolerance: 1119688132.0352123
  model = cd_fast.enet_coordinate_descent(
/Users/randallbloomquist/opt/an

{'regressor__alpha': 599.4842503189421, 'regressor__l1_ratio': 1.0}

In [88]:
y_predict = enet_cv.predict(X_test)
print('MSE: {:2e}'.format(mean_squared_error(y_test, y_predict)))

MSE: 2.216383e+09


In [89]:
calculate_five_percent(y_predict,y_test)

27.6

In [90]:
ridge = Ridge(alpha=1)
ridge.fit(X_train,y_train)
y_predict = ridge.predict(X_test)
print("score:", ridge.score(X_test, y_test))
print('MSE: {:2e}'.format(mean_squared_error(y_test, y_predict)))

score: 0.7723728171466584
MSE: 2.223683e+09


In [91]:
X_train.shape

(750, 10)

In [92]:
y_train.shape

(750,)

In [93]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_predict = lr.predict(X_test)
print("score:", lr.score(X_test, y_test))
print('MSE: {:2e}'.format(mean_squared_error(y_test, y_predict)))

score: 0.7722837384524026
MSE: 2.224553e+09


In [94]:
calculate_five_percent(y_predict,y_test)

28.000000000000004

In [21]:
np.arange(-5,5,1)

array([-5, -4, -3, -2, -1,  0,  1,  2,  3,  4])

In [22]:
type(y_train)

numpy.ndarray

In [23]:
kf = KFold()
skf = StratifiedKFold()
ridge_pipe = Pipeline([('scaler', StandardScaler()), ('regressor', Ridge())])
ridge_grid_params = {'regressor__alpha': np.arange(0,50,1)}
ridge_grid = GridSearchCV(ridge_pipe, ridge_grid_params, cv=kf, scoring = 'neg_mean_squared_error')
ridge_grid.fit(X_train, y_train)
print("Ridge Accuracy: {:.2e}".format(ridge_grid.score(X_test, y_test)))

Ridge Accuracy: -2.76e+09


In [24]:
kf = KFold()
skf = StratifiedKFold()
lasso_pipe = Pipeline([('scaler', StandardScaler()), ('regressor', Lasso())])
lasso_grid_params = {'regressor__alpha': np.arange(0.1,0.9,0.1)}
lasso_grid = GridSearchCV(lasso_pipe, lasso_grid_params, cv=kf, scoring = 'neg_mean_squared_error')
lasso_grid.fit(X_train, y_train)
print("Lasso Accuracy: {:.2e}".format(lasso_grid.score(X_test, y_test)))

Lasso Accuracy: -2.75e+09


In [32]:
kf = KFold()
skf = StratifiedKFold()
gbr_pipe = Pipeline([('scaler', StandardScaler()), ('regressor', GradientBoostingRegressor(loss='huber'))])
gbr_grid_params = {'regressor__alpha': np.arange(0.1,0.9,0.2),
                  'regressor__max_depth': np.arange(1,9,2),
                  'regressor__learning_rate': np.arange(0.01,0.2,0.03)}
gbr_grid = GridSearchCV(gbr_pipe, gbr_grid_params, cv=kf, scoring = 'neg_mean_squared_error')
gbr_grid.fit(X_train, y_train)
print("gbr Accuracy: {:.2e}".format(gbr_grid.score(X_test, y_test)))

gbr Accuracy: -1.66e+09


In [27]:
gbr_grid.best_params_

{'regressor__alpha': 0.4,
 'regressor__learning_rate': 0.16000000000000003,
 'regressor__max_depth': 4}

In [26]:
kf = KFold()
skf = StratifiedKFold()
gbr_pipe2 = Pipeline([('scaler', StandardScaler()), ('regressor', GradientBoostingRegressor(loss='huber'))])
gbr_grid_params2 = {'regressor__alpha': np.arange(0.1,0.9,0.3)}
gbr_grid2 = GridSearchCV(gbr_pipe2, gbr_grid_params2, cv=kf, scoring = 'neg_mean_squared_error')
gbr_grid2.fit(X_train, y_train)
print("gbr Accuracy: {:.2e}".format(gbr_grid.score(X_test, y_test)))

gbr Accuracy: -1.61e+09


In [30]:
gbr_grid2.best_params_

{'regressor__alpha': 0.7000000000000001}

In [183]:
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [184]:
np.arange(0,10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [95]:
X_df = pd.DataFrame(X)

In [96]:
X_df.corr() > 0.2

,0,1,2,3,4,5,6,7,8,9
0,True,False,False,False,False,False,False,False,False,False
1,False,True,True,False,True,True,False,True,False,False
2,False,True,True,False,True,False,False,True,False,False
3,False,False,False,True,False,False,False,False,False,False
4,False,True,True,False,True,True,False,True,False,False
5,False,True,False,False,True,True,False,True,False,False
6,False,False,False,False,False,False,True,False,False,False
7,False,True,True,False,True,True,False,True,False,False
8,False,False,False,False,False,False,False,False,True,False
9,False,False,False,False,False,False,False,False,False,True


In [97]:
X_train = sm.add_constant(X_train)

In [98]:
X_opt = X_train[:,np.arange(0,10)]
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.748
Method:                 Least Squares   F-statistic:                     248.1
Date:                Sat, 27 Mar 2021   Prob (F-statistic):          1.08e-216
Time:                        15:00:55   Log-Likelihood:                -9383.2
No. Observations:                 750   AIC:                         1.879e+04
Df Residuals:                     740   BIC:                         1.883e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.111e+07   3.74e+06     -2.971      0.003   -1.84e+07   -3.77e+06
x1            79.9889     46.982      1.703      0.089     -12.245     172.223
x2            94.9924      6.432     14.768      0.000      82.365     107.620
x3             6.4329      0.690      9.330      0.000       5.079       7.787
x4          1601.0958    287.692      5.565      0.000    1036.307    2165.885
x5          1319.0049     87.941     14.999      0.000    1146.361    1491.649
x6          7.166e+04    2.5e+04      2.871      0.004    2.27e+04    1.21e+05
x7          4888.7993   1.51e+04      0.323      0.747   -2.48e+04    3.46e+04
x8         -7844.5272   3576.188     -2.194      0.029   -1.49e+04    -823.844
x9           334.9652    770.976      0.434      0.664   -1178.595    1848.526
==============================================================================
Omnibus:                      767.532   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            77395.951
Skew:                           4.441   Prob(JB):                         0.00
Kurtosis:                      51.967   Cond. No.                     1.22e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.22e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [202]:
X_opt = X_train[:,[0,1,2,4,5,6,7,8,9,10]]

In [203]:
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.713
Method:                 Least Squares   F-statistic:                     2409.
Date:                Wed, 24 Mar 2021   Prob (F-statistic):               0.00
Time:                        22:03:59   Log-Likelihood:            -1.0857e+05
No. Observations:                8740   AIC:                         2.172e+05
Df Residuals:                    8730   BIC:                         2.172e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.625e+07   1.77e+06    -14.793      0.000   -2.97e+07   -2.28e+07
x1           -94.1917     11.221     -8.394      0.000    -116.188     -72.196
x2           108.0740      1.407     76.822      0.000     105.316     110.832
x3           831.0016     76.436     10.872      0.000     681.169     980.834
x4          1033.5446     21.627     47.789      0.000     991.150    1075.939
x5          8.316e+04   6871.368     12.103      0.000    6.97e+04    9.66e+04
x6         -1.834e+04   3989.132     -4.597      0.000   -2.62e+04   -1.05e+04
x7          1.941e+04   1819.451     10.669      0.000    1.58e+04     2.3e+04
x8          1133.9101    205.944      5.506      0.000     730.212    1537.608
x9          1.367e+04    753.404     18.142      0.000    1.22e+04    1.51e+04
==============================================================================
Omnibus:                     6564.327   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           786991.236
Skew:                           2.821   Prob(JB):                         0.00
Kurtosis:                      49.144   Cond. No.                     2.16e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.16e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [99]:
y_predict = regressor_OLS.predict(X_test)

In [100]:
print('MSE: {:.2e}'.format(mean_squared_error(y_test, y_predict)))

MSE: 7.55e+23


In [101]:
calculate_five_percent(y_predict,y_test)

0.0